In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
class Item:
    BASE_URL = "https://wiki.factorio.com"
    
    def __init__(self, name):
        self.name = name
        self.time = None
        self.count = 1
        self.recipe = []
    
    def parse_from_wiki(self):
        resp = requests.get(Item.BASE_URL + "/" + self.name)
        soup = BeautifulSoup(resp.text, 'lxml')
        self._get_recipe(soup)

    def _get_recipe(self, html_soup):
        recipe = html_soup.find("p", string=re.compile("Recipe*"))
        if not recipe:
            return
        recipe_table = recipe.parent.parent.parent
        for icon in recipe_table.find("td", class_='infobox-vrow-value').find_all("div", class_='factorio-icon'):
            r_name = icon.find("a")['href'][1:]
            r_count = icon.find("div", class_='factorio-icon-text').text

            if r_name == "Time":
                self.time = r_count
            elif r_name == self.name:
                self.count = r_count
            else:
                self.recipe.append((r_name, r_count))
    
    def __repr__(self):
        if self.time:
            return f"{self.name}\n  {self.time} sec -> {self.count}\n  {self.recipe}"
        else:
            return f"{self.name}"

In [3]:
class ItemSorage():
    def __init__(self):
        self.items = []
        
    def add(self, name, v=False):
        self.store_item(name, v)
            
    def get(self, name):
        for i in self.items:
            if i.name == name:
                return i
        return None
    
    def __contains__(self, name):
        return _has_name(name)
    
    def _has_name(self, name):
        for i in self.items:
            if i.name == name:
                return True
        return False
    
    def store_item(self, name, v=False, count=1, prefix="", last=False):
        if self._has_name(name):
            item = self.get(name)
        else:
            item = Item(name)
            item.parse_from_wiki()
            self.items.append(item)
        if v:
            print(prefix + name + f" ({count}x)")
        for i, (n, c) in enumerate(item.recipe):
            p = ""
            if len(prefix) != 0:
                if not last:
                    p = prefix[:-2] + "| "
                else:
                    p = prefix[:-2] + "  "

            is_last = (i == (len(item.recipe) - 1))
            if is_last:
                p = p + "|-"
            else:
                p = p + "+-"
            self.store_item(n, v=v, count=c, prefix=p, last=is_last)

In [4]:
storage = ItemSorage()

In [5]:
storage.add("Spidertron", v=True)

Spidertron (1x)
+-Efficiency_module_3 (2x)
| +-Advanced_circuit (5x)
| | +-Copper_cable (4x)
| | | |-Copper_plate (1x)
| | |   |-Copper_ore (1x)
| | +-Electronic_circuit (2x)
| | | +-Copper_cable (3x)
| | | | |-Copper_plate (1x)
| | | |   |-Copper_ore (1x)
| | | |-Iron_plate (1x)
| | |   |-Iron_ore (1x)
| | |-Plastic_bar (2x)
| |   +-Coal (1x)
| |   |-Petroleum_gas (20x)
| +-Efficiency_module_2 (5x)
| | +-Advanced_circuit (5x)
| | | +-Copper_cable (4x)
| | | | |-Copper_plate (1x)
| | | |   |-Copper_ore (1x)
| | | +-Electronic_circuit (2x)
| | | | +-Copper_cable (3x)
| | | | | |-Copper_plate (1x)
| | | | |   |-Copper_ore (1x)
| | | | |-Iron_plate (1x)
| | | |   |-Iron_ore (1x)
| | | |-Plastic_bar (2x)
| | |   +-Coal (1x)
| | |   |-Petroleum_gas (20x)
| | +-Efficiency_module (4x)
| | | +-Advanced_circuit (5x)
| | | | +-Copper_cable (4x)
| | | | | |-Copper_plate (1x)
| | | | |   |-Copper_ore (1x)
| | | | +-Electronic_circuit (2x)
| | | | | +-Copper_cable (3x)
| | | | | | |-Copper_plate (1

In [6]:
storage.get("Copper_cable")

Copper_cable
  0.5 sec -> 2
  [('Copper_plate', '1')]

In [7]:
storage.get("Chemical_science_pack")